In [1]:
import gymnasium as gym
import ale_py
gym.register_envs(ale_py)
gym.__version__

'1.0.0'

In [2]:
import time
env = gym.make("ALE/Breakout-v5", render_mode="human")
print(env.action_space.sample())
observation, info = env.reset(seed=42)
for _ in range(100):
    time.sleep(0.1)
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

env.close()

A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]


3
